In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/Face_Recognition

/content/drive/.shortcut-targets-by-id/1-7XeLp7el2PwfO1ASYbDd4MwTAeC52oX/Face_Recognition


In [3]:
path='/content/drive/My Drive/Colab Notebooks/Face_Recognition'

In [4]:
! pip install gdown

!pip install twilio
!pip install redmail

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.8 MB/s eta 0:00:00


In [5]:
# Libraries used

# Usual Libraries
import os
import re
import glob
from google.colab.patches import cv2_imshow
from datetime import datetime

# For Face Detection
import cv2
import matplotlib.pyplot as plt
import dlib

# Tensorflow Library Mainly for keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

# Threading for making the testing faster
import threading
from collections import defaultdict

import pandas as pd

# For Sending the attendance Information through Mass Media like Whatsapp and Gmail.

#For Whatsapp
from twilio.rest import Client

#For Gmail
from redmail import outlook


In [6]:
# Get Image names stored in "Images" folder
image_path_names=[]
person_names=set()
for file_name in glob.glob(path+'/Images/*[1-9]*.jpg'):
  image_path_names.append(file_name)
  person_names.add(re.sub(r'[0-9]', '', image_path_names[-1].split('/')[-1].split('.')[0]))

In [7]:
#Total Number of Student Name
print("Total Number of students faces are {0}".format(len(image_path_names)))

Total Number of students faces are 78


In [8]:
# All the names of Students
print("All Student names",person_names)

All Student names {'MalliReddy', 'Pragna', 'RamCharanTeja', 'Ranjith', 'NTR', 'Shiva', 'saiCharanTeja', 'Pragna ()'}


  There are total 70 images containing 10 images per person.

In [9]:
# Download Dlib CNN face detector
! wget http://dlib.net/files/mmod_human_face_detector.dat.bz2

--2023-07-14 15:57:26--  http://dlib.net/files/mmod_human_face_detector.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694709 (678K)
Saving to: ‘mmod_human_face_detector.dat.bz2’

mmod_human_face_det 100%[===================>] 678.43K  --.-KB/s    in 0.1s    

2023-07-14 15:57:26 (5.37 MB/s) - ‘mmod_human_face_detector.dat.bz2’ saved [694709/694709]



In [10]:
!bzip2 -dk mmod_human_face_detector.dat.bz2

bzip2: Output file mmod_human_face_detector.dat already exists.


In [11]:
%rm mmod_human_face_detector.dat.bz2

In [12]:
# Load CNN face detector into dlib
dnnFaceDetector=dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

In [13]:
# Create a Folder Image_Crop
if(os.path.exists(path+'/Images_crop/')== False):
    os.mkdir(path+'/Images_crop/')

In [14]:
# For each person create a separate folder
for person in person_names:
    if(os.path.exists(path+'/Images_crop/'+person+'/') == False):
        os.mkdir(path+'/Images_crop/'+person+'/')

In [15]:
def Crop_Detected_Faces(file_name,img_path):
    img=cv2.imread(file_name)
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    rects=dnnFaceDetector(gray,1)

    left,top,right,bottom=0,0,0,0
    for (i,rect) in enumerate(rects):
        left=rect.rect.left() #x1
        top=rect.rect.top() #y1
        right=rect.rect.right() #x2
        bottom=rect.rect.bottom() #y2
    width=right-left
    height=bottom-top
    img_crop=img[abs(top):abs(top)+height,abs(left):abs(left)+width]

    cv2.imwrite(img_path,img_crop)


In [16]:
# Detect face, crop detected face and save them in corresponding person folder
threads = []
for file_name in image_path_names:
    img_path=path+'/Images_crop/'+re.sub(r'[0-9]', '', file_name.split('/')[-1].split('.')[0])+"/"+file_name.split('/')[-1]
    t = threading.Thread(target=Crop_Detected_Faces, args=(file_name,img_path,))

    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

In [17]:
# Get Image names for testing
test_image_path_names=[]
for file_name in glob.glob(path+'/Images_test/*_[123].jpg'):
  test_image_path_names.append(file_name)

In [18]:
print("Total number of student test images {0}".format(len(test_image_path_names)))

Total number of student test images 21


For each person 3 images to test in Images_test folder.

In [19]:
# Creating a Test Images Crop folder
if(os.path.exists(path+'/Images_Test_crop/') == False):
    os.mkdir(path+'/Images_Test_crop/')

In [20]:
# Create Separate folder for each person in "Test_Images_crop" folder
for person in person_names:
    if(os.path.exists(path+'/Images_Test_crop/'+person+'/') == False):
        os.mkdir(path+'/Images_Test_crop/'+person+'/')

In [21]:
# Detect face,crop face and save in corresponding folder
threads = []
for file_name in test_image_path_names:
    img_path=path+'/Images_Test_crop/'+file_name.split('/')[-1].split('_')[0]+'/'+file_name.split('/')[-1]
    t = threading.Thread(target=Crop_Detected_Faces, args=(file_name,img_path))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

<pre>Here images are sorted to corresponding test and train folders of same person
Directory structure :
|Images /
|  |-- (60 images) - Images of student. 10 Images of each student.
|Images_crop /
|  |--SaiCharanTeja /
|     |--(10 images)
|  |--Pargna /
|     |--(10 images)
|  |--MalliReddy /
|         |--(10 imgaes)
|  |--Pranay /
|         |--(10 imgaes)
|Images_test / - Given Images to test the model.Each of 3 Images
|  |-- .. / (18 images)
|Images_test_crop /
|  |--SaiCharanTeja / (3 images)
|  |--Pargna / (3 images)
|  |--MalliReddy / (3 images)
|  |--Pranay / (3 images)
|Test_Images / - Images  to be tested.
|  |-- There is no specified Number of Images.
|  |--Predictions / - The predicted Images are stored with in this folder.
|  |--Attendance/ - To store the daily attendance
|vgg_face_weights.h5
|train_data.npy
|train_labels.npy
|test_data.npy
|test_labels.npy
|Face_Recognition.ipynb
|mmod_human_face_detector.dat
|face_classifier_model.h5
</pre>

In [22]:
%ls

face_classifier_model.h5      Predictions/        test_labels.npy
Images/                       StudentAttendance/  train_data.npy
Images_crop/                  Students_Info.xlsx  train_labels.npy
Images_test/                  test_data.npy       vgg-face-keras-fc.h5
Images_Test_crop/             Test_Images/        vgg_face_weights.h5
mmod_human_face_detector.dat  Test_Images_crop/


In [23]:
#Define VGG_FACE_MODEL architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [24]:
# Load VGG Face model weights
model.load_weights(path+'/vgg_face_weights.h5')

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                        

In [26]:
# Remove Last Softmax layer and get model upto last flatten layer with outputs 2622 units
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)

In [27]:
#Prepare Training Data
x_train=[]
y_train=[]
person_folders=os.listdir(path+'/Images_crop/')

person_rep=dict()
for i,person in enumerate(person_folders):
  person_rep[i]=person
  image_names=os.listdir('Images_crop/'+person+'/')
  for image_name in image_names:
    img=load_img(path+'/Images_crop/'+person+'/'+image_name,target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=preprocess_input(img)
    img_encode=vgg_face(img)
    x_train.append(np.squeeze(K.eval(img_encode)).tolist())
    y_train.append(i)

In [28]:
person_rep

{0: 'Shiva',
 1: 'RamCharanTeja',
 2: 'saiCharanTeja',
 3: 'Ranjith',
 4: 'NTR',
 5: 'Pragna ()',
 6: 'MalliReddy',
 7: 'Pragna'}

In [29]:
x_train=np.array(x_train)
y_train=np.array(y_train)

In [30]:
#Prepare Test Data
x_test=[]
y_test=[]
person_folders=os.listdir(path+'/Images_Test_crop/')
for i,person in enumerate(person_folders):
  image_names=os.listdir('Images_Test_crop/'+person+'/')
  for image_name in image_names:
    img=load_img(path+'/Images_Test_crop/'+person+'/'+image_name,target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)

    img=preprocess_input(img)
    img_encode=vgg_face(img)

    x_test.append(np.squeeze(K.eval(img_encode)).tolist())
    y_test.append(i)

In [31]:
x_test=np.array(x_test)
y_test=np.array(y_test)

In [32]:
# Save test and train data for later use
np.save('train_data',x_train)
np.save('train_labels',y_train)
np.save('test_data',x_test)
np.save('test_labels',y_test)

In [33]:
# Load saved data
x_train=np.load('train_data.npy')
y_train=np.load('train_labels.npy')
x_test=np.load('test_data.npy')
y_test=np.load('test_labels.npy')

In [65]:
# Softmax regressor to classify images based on encoding
classifier_model=Sequential()

classifier_model.add(Dense(units=100,input_dim=x_train.shape[1],kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.3))

classifier_model.add(Dense(units=10,kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.2))

classifier_model.add(Dense(units=8,kernel_initializer='he_uniform'))
classifier_model.add(Activation('softmax'))
classifier_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),optimizer='nadam',metrics=['accuracy'])
#classifier_model.compile(loss=tf.keras.losses.binary_crossentropy(),optimizer='Adam',metrics=['accuracy'])
# classifier_model.compile(loss="sparse_categorical_crossentropy",optimizer='Nadam',metrics=['accuracy'])

In [66]:
classifier_model.fit(x_train,y_train,epochs=250,validation_data=(x_test,y_test))

Epoch 1/250


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


3/3 [==============================] - 4s 117ms/step - loss: 1.9997 - accuracy: 0.2179 - val_loss: 3.0708 - val_accuracy: 0.0476
Epoch 2/250
3/3 [==============================] - 0s 32ms/step - loss: 0.9403 - accuracy: 0.7564 - val_loss: 3.2728 - val_accuracy: 0.0000e+00
Epoch 3/250
3/3 [==============================] - 0s 23ms/step - loss: 0.7714 - accuracy: 0.8590 - val_loss: 3.3401 - val_accuracy: 0.0000e+00
Epoch 4/250
3/3 [==============================] - 0s 32ms/step - loss: 0.5858 - accuracy: 0.9744 - val_loss: 3.3153 - val_accuracy: 0.0000e+00
Epoch 5/250
3/3 [==============================] - 0s 23ms/step - loss: 0.6131 - accuracy: 0.9615 - val_loss: 3.3905 - val_accuracy: 0.0952
Epoch 6/250
3/3 [==============================] - 0s 24ms/step - loss: 0.5992 - accuracy: 0.9615 - val_loss: 3.3500 - val_accuracy: 0.0476
Epoch 7/250
3/3 [==============================] - 0s 33ms/step - loss: 0.5223 - accuracy: 0.9487 - val_loss: 3.3021 - val_accuracy: 0.0952
Epoch 8/250
3/3 [==

In [67]:
# Save model for later use
tf.keras.models.save_model(classifier_model,'/content/drive/My Drive/Colab Notebooks/Face_Recognition/face_classifier_model.h5')

In [68]:
# Load saved model
classifier_model=tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/Face_Recognition/face_classifier_model.h5')

In [69]:
# Path to folder which contains images to be tested and predicted
test_images_path=path+'/Test_Images/'

In [70]:
dnnFaceDetector=dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

In [71]:
def plot(img):
  plt.figure(figsize=(8,4))
  plt.imshow(img[:,:,::-1])
  plt.show()

In [41]:
from zmq.sugar import frame
video_path = "/content/drive/MyDrive/videos/Natu_Natu.mp4"
output_path = "/content/drive/MyDrive/Colab Notebooks/Face_Recognition/Test_Images"


vidcap = cv2.VideoCapture(video_path)
success, image = vidcap.read()
count = 0
frameNum =0
while success:
    if count % 1000 == 0:
        cv2.imwrite(output_path + "/frame%d.jpg" %frameNum , image)
        frameNum += 1
    success, image = vidcap.read()
    count += 10

In [ ]:
# Label names for class numbers
# person_rep={0: 'Ranjith', 1: 'Shiva', 2: 'saiCharanTeja',3:"None"}

In [42]:
if(os.path.exists(path+'/Predictions')==False):
    os.mkdir(path+'/Predictions')

In [43]:
if(os.path.exists(path+'/Test_Images_crop/') == False):
    os.mkdir(path+'/Test_Images_crop/')

In [72]:
# Function to predict faces in a name and update the frequency dictionary

def process_name(img_name, frequency_dict):
  try:
    img=cv2.imread(path+'/Test_Images/'+img_name)
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect Faces
    rects=dnnFaceDetector(gray,1)
    left,top,right,bottom=0,0,0,0

    for (i,rect) in enumerate(rects):
        # Extract Each Face
        left=rect.rect.left() #x1
        top=rect.rect.top() #y1
        right=rect.rect.right() #x2
        bottom=rect.rect.bottom() #y2
        width=right-left
        height=bottom-top
        img_crop=img[abs(top):abs(top)+height,abs(left):abs(left)+width]
        cv2.imwrite(path+'/Test_Images_crop/'+img_name[:-5]+"Crop_img.jpg",img_crop)
        # Get Embeddings
        crop_img=cv2.imread(path+'/Test_Images_crop/'+img_name[:-5]+"Crop_img.jpg")

        crop_img= cv2.resize(crop_img,(224,224))

        crop_img=img_to_array(crop_img)
        crop_img=np.expand_dims(crop_img,axis=0)
        crop_img=preprocess_input(crop_img)
        img_encode=vgg_face(crop_img)


        # Make Predictions
        embed=K.eval(img_encode)
        person=classifier_model.predict(embed)
        name=person_rep[np.argmax(person)]

        os.remove(path+'/Test_Images_crop/'+img_name[:-5]+"Crop_img.jpg")

        cv2.rectangle(img,(abs(left),abs(top)),(right,bottom),(0,255,0), 2)

        if(np.max(person)>0.75):
            img=cv2.putText(img,name,(abs(left),abs(top)-10),cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,255),2,cv2.LINE_AA)
            img=cv2.putText(img,str(np.max(person)),(right,bottom+10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,cv2.LINE_AA)
            # Update the frequency dictionary
            frequency_dict[name] +=1

    # Save images with bounding box,name and accuracy
    cv2.imwrite(path+'/Predictions/'+img_name,img)

  except:
    pass


In [73]:
# List of names to process
names = list(person_names)

# Dictionary to store the frequency of faces in each name
frequency_dict = defaultdict(int)

for i in names:
    frequency_dict[i] =0

# Create a lock to synchronize access to the dictionary
lock = threading.Lock()

# Create threads to process each name
threads = []


for img_name in os.listdir('Test_Images/'):
    t = threading.Thread(target=process_name, args=(img_name, frequency_dict))
    threads.append(t)
    t.start()


# Wait for all threads to finish
for t in threads:
    t.join()

# Print the frequencies of faces in each name
for name, frequency in frequency_dict.items():
    print(f"Name: {name}, Frequency: {frequency}")
attendance = frequency_dict

1/1 [==============================] - 0s 57ms/step
Name: MalliReddy, Frequency: 7
Name: Pragna, Frequency: 0
Name: RamCharanTeja, Frequency: 10
Name: Ranjith, Frequency: 0
Name: NTR, Frequency: 2
Name: Shiva, Frequency: 5
Name: saiCharanTeja, Frequency: 12
Name: Pragna (), Frequency: 4


In [64]:
class Student:
    def __init__(self, name, MobileNumber,Mail):
        self.name = name
        self.MobileNumber = MobileNumber
        self.Mail = Mail

In [ ]:
#load Student Info details
df = pd.read_excel(path+'/Students_Info.xlsx',index_col=0);
students_Info =[]
for idx in df.index:
    students_Info.append(Student(df["Name"][idx],df["Mobile_Number"][idx],df["Mail_Id"][idx]))

In [ ]:
for i in students_Info:
    print(i.name,i.MobileNumber,i.Mail)

In [ ]:
if(os.path.exists(path+'/StudentAttendance/')== False):
    os.mkdir(path+'/StudentAttendance/')

In [ ]:
# datetime object containing current date and time
now = datetime.now()

# dd/mm/YY H:M:S
dt_string = now.strftime("D_%d_%m_%Y_T_%H_%M_%S.xlsx")

dict_ ={"Name":attendance.keys(),"Attentiveness":attendance.values()}

df = pd.DataFrame(dict_)
df.to_excel(path+'/StudentAttendance/'+dt_string)
df = pd.read_excel(path+'/StudentAttendance/'+dt_string,index_col=0)
df

In [ ]:
def SendMessage(message,mobileNumber):
    account_sid = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    auth_token = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

    client = Client(account_sid, auth_token)
    message = client.messages.create(
    body=message,
    from_="whatsapp:+XXXXXXXX",
    to="whatsapp:+"+str(mobileNumber)
    )

In [ ]:
for i in students_Info:
    attentiveness = 0
    if(i.name in attendance.keys()):
        attentiveness = attendance[i.name]

    print("Message Successfully sent to :" +i.name)
    msg ="Hello : "+i.name+ ",\n Your attentiveness Percentage is "+str(attentiveness)+" %"
    if(attentiveness <50):
         msg = msg + "\n Make sure to Focus on the class Next time."
         msg = msg + ".\nAnd You are Absent for the class Today\n"
    else:
         msg = msg + ". \nAnd You are Present for the class Today.\n"
    msg+="Regards JNTUHCES BATCH 7"
    SendMessage(msg,i.MobileNumber)


In [ ]:
outlook.username = "XXXXXXX@outlook.com"
outlook.password = "XXXXXXXX"

def sendmail(candidate_email,attentive,attentiveness):
    status = "PRESENT"
    header = "This is "
    footer ="Your active participation and attentiveness were commendable, and we appreciate your dedication to the class.</p><p>Keep up the excellent work!</p>"
    if attentive == False:
        status = "ABSENT"
        header = 'We regret '
        footer =" We kindly request that you make a conscious effort to be more attentive during future classes.</p><p>Keep up the excellent work!</p>"

    message = '''
        <!DOCTYPE html>
<head>
<style>
    body {
        background-image: url('https://jntuh.ac.in/images/jntuhlogo.png');
        background-size: cover;
        background-repeat: no-repeat;
        background-color: rgba(20, 136, 234, 0.2);
        opacity:1;
        z-index: -1;
    }

    #card {
        position: relative;
        top:60px;
        width: 320px;
        display: block;
        margin: auto;
        text-align: center;
        font-family: 'Source Sans Pro', sans-serif;
        border: 6px solid  #0000FF;
        border-radius: 10px;


    }

    #upper-side {
        padding: 2em;
        background-color: rgba(0, 0, 255, 1);
        display: block;
        color: #fff;

        border-bottom-right-radius: 10px;
        border-bottom-left-radius: 10px;

    }


    #status {
        font-weight: lighter;
        text-transform: uppercase;
        letter-spacing: 5px;
        font-size: 1em;
        margin-top: -0.2em;
        margin-bottom: 0;
    }

    #lower-side {
        padding: 2em 2em 5em 2em;
        background: #fff;
        display: block;
        border-bottom-right-radius: 10px;
        border-bottom-left-radius: 10px;
    }

    #message {
        margin-top: -0.5em;
        color: #757575;
        letter-spacing: 1px;
        color: #000000;
  		font-size: 16px;
  		font-weight: bold;
  		font-style: normal;

    }
.envelope-header {
      margin-bottom: 30px;
       text-align: center;
    }

</style>

</head>

<html>
<body>
<div class="envelope-header">
      <img src="https://jntuh.ac.in/images/jntuhlogo.png" alt="Attendance">
    </div>
<div id="card" class="animated fadeIn">

    <div id="upper-side">
        <h1 id="status"><b>Attendance Status</b></h1>
    </div>
    <div id="lower-side">
        <p id="message">'''+ header +'''to inform you that you were marked <h3> <b style="color: #FF0000;">'''+ status+''' </b> </h3>
        </p>
        <p id="message">'''+footer+'''
        <p style="color: #006400;font-weight: bold; "><b>And Your percentage of Attentativeness is</b>
        	<h1>
              	<b style="color: #00FF00;">'''+ str(attentiveness) +'''%
            	<b>
             </h1>
        </p>
  	</div>
</div>

</body>
</html>

'''
    outlook.send(
        receivers=[candidate_email,],
        subject="Your Attendance @ JNTU",
        html=message
    )

In [ ]:
#Mailing
for i in students_Info:
    status = False
    attentiveness = 0
    if(i.name in attendance.keys()):
        if(attendance[i.name] > 50):
            status = True
        attentiveness = attendance[i.name]
    sendmail(i.Mail,status,attentiveness)